In [ ]:
# 导入需要的包
# Import the required packages.
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np
import random

# 创建数据加载器
# 真实数据label为0，生成图像label为1
# Create a data loader
# Real data label is 0, generated image label is 1

train_dir = '/bohr/train-9zgj/v1/'  #训练集地址 #Address of dataset
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        for label, sub_dir in enumerate(os.listdir(root_dir)):
            print(label)  #只需要把label都打出来就可以看出来这段代码是什么意思 #Just print out all the labels and you can see what this code means.
            full_dir = os.path.join(root_dir, sub_dir)
            for img_name in os.listdir(full_dir):
                img_path = os.path.join(full_dir, img_name)
                self.image_paths.append(img_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# 数据预处理 Data preprocessing
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = CustomDataset(train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


# 训练函数 Training function
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    max_accuracy = 0 # 打印最高准确率 Print the highest accuracy.
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        train_accuracy = eval_model(model,train_loader)    
        log_message = f'Epoch {epoch+1}/{num_epochs}, Xiao Ai Train Accuracy: {train_accuracy:.4f}'
        print(log_message)
    #print("max_accuracy:", max_accuracy)

# 评估函数 Evaluation function.
def eval_model(model, data_loader):
    model.eval()
    corrects = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)
            outputs = model(inputs)
            preds = outputs >= 0.5
            corrects += torch.sum(preds == labels).item()
            total += labels.size(0)
    accuracy = corrects / total
    return accuracy

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 10, 5)
        self.fc1 = nn.Linear(10 * 5 * 5, 70)
        self.fc2 = nn.Linear(70, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 10 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x


# 设置设备 Set up device.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# 创建模型、损失函数和优化器 Create model, loss function, and optimizer.
model = MyModel().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
epochs = 5 
#Train the model and log the process.
train_model(model, train_loader, criterion, optimizer, num_epochs= epochs)


# 请将你的model代码(包括需要import的先导module，例如torch和torch.nn）写在下方以生成一个model的结构文件方便批卷时读取
# Please write your model code (including the necessary preceding modules to import, such as torch and torch.nn) below to generate a model structure file for easy reading during scoring.
model_code = """  
import torch
import torch.nn as nn
import torch.optim as optim
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 10, 5)
        self.fc1 = nn.Linear(10 * 5 * 5, 70)
        self.fc2 = nn.Linear(70, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 10 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x
"""
# 将以上模型代码写入文件 # Write the model code into a file.
with open('submission_model.py', 'w') as f:
    f.write(model_code)
print("submission_model.py file is generated.")
torch.save(model.state_dict(), 'submission_dic.pth')

In [ ]:
import zipfile
import os

# 定义要打包的文件和压缩文件名 
# Define the files to be packaged and the compressed file name.
files_to_zip = ['submission_model.py', 'submission_dic.pth']
zip_filename = 'submission.zip'

# 创建一个 zip 文件, Create a zip file
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in files_to_zip:
        # 将文件添加到 zip 文件中，Add files to the zip file.
        zipf.write(file, os.path.basename(file))

print(f'{zip_filename} is created successfully!')

In [ ]:
!cp submission.zip /personal  
#通过这条指令可以在bohrium的主页中的左侧“文件”中可以该.zip文件并下载
#This instruction allows you to find and download the .zip file in the “Files” section on the left side of the Bohrium homepage.